# Process

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

model_cb = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='F1',
    depth=8,
    learning_rate=0.05,
    iterations=500,
    random_seed=42,
    verbose=100
)

model_cb.fit(X_train, y_train, eval_set=(X_val, y_val))

y_pred = model_cb.predict(X_val)

print("F1:", f1_score(y_val, y_pred, average='binary'))


In [ ]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42,
    n_jobs=-1
)

model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_val)

print("XGBoost F1:", f1_score(y_val, y_pred_xgb, average='binary'))